# Unsupervised Learning Predict
© Explore Data Science Academy

---
*   Thomas Kenyon
*   Name
*   Name
*   Name
*   Name
---

### Honour Code

We, Team 8, confirm - by submitting this document - that the solutions in this notebook are a result of our own work and that we abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

---

### Predict Overview: Movie Recommender

--Insert description and context here

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Data Engineering</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>

<a href=#seven>7. Model Explanations</a>

<a href=#eight>8. Appendix</a>

In [ ]:
# Import our regular old heroes 
import numpy as np
import pandas as pd
import scipy as sp # <-- The sister of Numpy, used in our code for numerical efficientcy. 
import matplotlib.pyplot as plt
import seaborn as sns

# Entity featurization and similarity computation
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import TfidfVectorizer

# Libraries used during sorting procedures.
import operator # <-- Convienient item retrieval during iteration 
import heapq # <-- Efficient sorting of large lists

# Imported for our sanity
import warnings
warnings.filterwarnings('ignore')

In [ ]:
genome_scores = pd.read_csv('genome_scores.csv')
genome_tags = pd.read_csv('genome_tags.csv')
imdb_data = pd.read_csv('imdb_data.csv')
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
tags = pd.read_csv('tags.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

<a id="four"></a>
## 4. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---

Prior to training a model with any sort of data, it is essential to re-engineer it. This is to ensure that the data is in a consistent form with no missing values, incorrect data types or just plain incorrect data. For structured,  numeric data, this entails scaling values, filling in missing values and typecasting any non-numeric data to numeric form. Non-numeric data, such as this database is often unstructured and consists of text data. This type of data is not easily interpretable by computers. Computers work with 1s and 0s, not letters and words. Therefore it must be converted into a form that is interpretable, and therefore, numeric data. 

### Content-based Recommender

---
First thing, we need to merge the movies dataframe with the imdb_data dataframe, these two dataframes contain most of the information we need to construct our recommender.

As you can see, for each movie we now have multiple columns that contain useful information about each movie:
* genres
* title_cast
* director
* plot_keywords

There are more columns that could be useful, such as budget, but lets stick to these for now.

In [ ]:
joined2 = movies.merge(imdb_data,  how='left', on = 'movieId')
joined2.head()

There is a lot of actors names for each movie in the title_cast column! Since we're going to ultimately be vectorizing all of this data, we should ask ourselves if we really need all the actors for each movie listed in this column. For the sake of minimizing the number of features in our vectorized dataset, lets reduce this to the top 3 actors. The first 3 actors in this column seem to be the top-billed/most prominent cast for each film, so it's likely that they're the most important anyway.

In [ ]:
def first_3(actors):
  s = []
  s = [actor for actor in actors if len(s) < 3]
  s2 = s[:3]
  return '|'.join([actor for actor in s2])

# x = ['Rhys Ifans', 'Tim Allen', 'Don Rickles', 'Crispin Glover', 'Christian McKay']
# print(first_3(x))


joined2['title_cast'] = joined2['title_cast'].apply(lambda x: str(x).split('|'))
joined2['title_cast'] = joined2['title_cast'].apply(first_3)
joined2.head(2)

One problem with this dataset are the number of nan values. Most of these are actual np.nan null values, however some of them are actually 'nan' strings. Sneaky! We'll need to convert all these cryptic nan values to real nan values and then imput something whenever they occur, since null values cannot be passed to a vectorizer.

In [ ]:
def check_nan(field):
  '''Some NaN values are being stored as strings, so fillna wont work'''
  if not isinstance(field, str):
    return np.nan
  f2 = field.strip().lower()
  if 'nan' in f2:
    return np.nan
  else:
    return field

joined3 = joined2.copy()
joined3['title_cast'] = joined3['title_cast'].apply(check_nan)
joined3['plot_keywords'] = joined3['plot_keywords'].apply(check_nan)

Lets replace all nan values with empty strings. This means that there will be a lot of movies with a lot of empty metadata columns, but hopefully when we include the genome tag dataset this will become less of an issue.

In [ ]:
joined3['director'].fillna('', inplace = True)
joined3['title_cast'].fillna('', inplace = True)
joined3['runtime'].fillna(0, inplace = True)
joined3['budget'].fillna(0, inplace = True)
joined3['plot_keywords'].fillna('', inplace = True)

Creating a word soup column. This column has all the text data in it that we will vectorize. Think of all the words/names in these columns becoming tags. I've added director 3 times so that it has more weighting. IE, if someone likes a movie like Inception, then they are likely to enjoy other movies directer by Christopher Nolan. This is quite a janky way of altering the weightings of specific tags, but creating a word soup like this does simplify the vectorizing process.

In [ ]:
joined3['soup'] = joined3['genres'] + '|' + joined3['director'] + '|' + joined3['director'] + '|' + joined3['director'] + '|' + joined3['plot_keywords'] + '|'+ joined3['title_cast']
joined3.iloc[1]

The movie titles contain their release year. I remove that and put the years in their own column using the two cells below

In [ ]:
def get_year(title):
  s = title.split()
  year = s[-1]
  if '(' in year and ')' in year:
    year2 = year[1:-1]
    return year2
  else:
    return np.nan
  
joined3['year'] = joined3['title'].apply(get_year)